In [2]:
# To access Google Drive:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
REVIEW_PATH = "/content/gdrive/MyDrive/reviews.csv"

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [5]:
BATCH_SIZE=8

In [6]:
data = pd.read_csv(REVIEW_PATH)

In [7]:
data.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOEhZuqSqqWnaKRgv-9ABYdajFUB0WugPGh-SG-...,Eric Tie,https://play-lh.googleusercontent.com/a-/AOh14...,I cannot open the app anymore,1,0,5.4.0.6,2020-10-27 21:24:41,NaN,NaN,newest,com.anydo
1,gp:AOqpTOH0WP4IQKBZ2LrdNmFy_YmpPCVrV3diEU9KGm3...,john alpha,https://play-lh.googleusercontent.com/a-/AOh14...,I have been begging for a refund from this app...,1,0,NaN,2020-10-27 14:03:28,"Please note that from checking our records, yo...",2020-10-27 15:05:52,newest,com.anydo
2,gp:AOqpTOEMCkJB8Iq1p-r9dPwnSYadA5BkPWTf32Z1azu...,Sudhakar .S,https://play-lh.googleusercontent.com/a-/AOh14...,Very costly for the premium version (approx In...,1,0,NaN,2020-10-27 08:18:40,NaN,NaN,newest,com.anydo
3,gp:AOqpTOGFrUWuKGycpje8kszj3uwHN6tU_fd4gLVFy9z...,SKGflorida@bellsouth.net DAVID S,https://play-lh.googleusercontent.com/-75aK0WF...,"Used to keep me organized, but all the 2020 UP...",1,0,NaN,2020-10-26 13:28:07,What do you find troublesome about the update?...,2020-10-26 14:58:29,newest,com.anydo
4,gp:AOqpTOHls7DW8wmDFzTkHwxuqFkdNQtKHmO6Pt9jhZE...,Louann Stoker,https://play-lh.googleusercontent.com/-pBcY_Z-...,Dan Birthday Oct 28,1,0,5.6.0.7,2020-10-26 06:10:50,NaN,NaN,newest,com.anydo


In [8]:
data = data[["content","score"]].rename(columns={"content":"text","score":"label"})

In [9]:
data.head()

,text,label
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 2020 UP...",1
4,Dan Birthday Oct 28,1


In [10]:
data['label'] = data['label'].replace([1,2,3,4,5],[0,1,2,3,4])

In [11]:
data['label'].value_counts()

label
4    2879
3    2775
0    2506
1    2344
2    1991
Name: count, dtype: int64

In [12]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


## **Train Test Split**

In [13]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2)


In [14]:
from datasets import DatasetDict, Dataset

In [15]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Create a DatasetDict to store both train and test datasets
dataset= DatasetDict({'train': train_dataset, 'test': test_dataset})


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 9996
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2499
    })
})

## **Loading Tokenizer**

<!-- **Loading a pre-trained BERT Model** -->

<!-- ** -->

In [17]:
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)
from transformers import TFAutoModel, AutoTokenizer

In [18]:
# model_id="nlptown/bert-base-multilingual-uncased-sentiment"
# tokenizer = BertTokenizerFast.from_pretrained(model_id)
tokenizer =  AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

#### **Passing review data into tokenizer**


In [19]:
output=tokenizer(dataset['train'][:2]['text'],padding=True,truncation=True,max_length=128)
# # print(output)


**Checking the decoded version to tokenized data**

Here the [CLS] shows the start of the sentence and [SEP] shows the end of the sentence [PAD] is added to end the seqence to make it reach the maximum length. in other words it ensure that all of the sequence are of the same length

In [20]:
tokenizer.decode(output['input_ids'][0])

'[CLS] it was good until they updated it and took out the today, someday, upcoming categories out, which is how i organized everything [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

**Applying to whole dataset**

In [21]:
def preprocess_function(examples):
  return tokenizer(examples["text"],padding=True,truncation=True,max_length=128)

In [22]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/9996 [00:00<?, ? examples/s]

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9996
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2499
    })
})

### **Creating tensorflow dataset from tokenized dataset**

In [23]:
tf_train_datasets = tokenized_dataset["train"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [24]:
tf_val_datasets = tokenized_dataset["test"].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
    #collate_fn=data_collator
)

Separating the Labels from the dataset

In [25]:
def swap_positions(dataset):
  return {'input_ids':dataset['input_ids'],
          'token_type_ids':dataset['token_type_ids'],
          'attention_mask':dataset['attention_mask'],},dataset['label']

In [26]:
tf_train_dataset=tf_train_datasets.map(swap_positions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset=tf_val_datasets.map(swap_positions).prefetch(tf.data.AUTOTUNE)

## **Modelling Using TFBertForSequenceClassification**

In [27]:
# from transformers import AutoModelForSequenceClassification

In [28]:
# model=TFBertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment",num_labels=5)
model = TFAutoModel.from_pretrained("bert-base-uncased")
model.summary()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109482240 (417.64 MB)
Trainable params: 109482240 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        # self.drop = tf.keras.layers.Dropout(rate=0.3)
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        # pooled_output = outputs[1]  # Using [CLS] token output
        # output = self.dropout(pooled_output)
        return self.fc(x)

### **Training**

In [30]:
# num_epochs = 5
# batches_per_epoch = len(tokenized_dataset["train"]) // BATCH_SIZE
# total_train_steps = int(batches_per_epoch * num_epochs)

In [31]:
# optimizer, schedule = create_optimizer(init_lr=2e-5,num_warmup_steps=0, num_train_steps=total_train_steps)

In [32]:
# print(optimizer)

In [33]:
classifier = BERTForClassification(model, num_classes=5)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history=classifier.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=3)

Epoch 1/3
 232/1250 [====>.........................] - ETA: 3:15:08 - loss: 1.7368 - accuracy: 0.2064

In [ ]:
# model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(),
# optimizer=optimizer,
#     metrics=['accuracy'],)
#     #run_eagerly=True)

#     # CategoricalCrossentropy

In [ ]:
# history=model.fit(
#     tf_train_dataset,
#     validation_data=tf_val_dataset,
#     epochs=5)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
classifier.evaluate(tf_val_dataset)